In [2]:
import os
import fnmatch
import shutil

# ==============================================================================
# CONFIGURATION - CHANGE THESE VALUES
# ==============================================================================

# 1. The full path to the folder you want to process.
#    On Windows: r"C:\Users\YourUser\Documents\MyFolder"
#    On macOS/Linux: "/home/youruser/documents/my_folder"
target_folder = r"F:\OneDrive - Green Energy\Desktop\samples\Flutter" # <--- SET YOUR SOURCE FOLDER

# 2. The path where the converted .txt files will be saved.
#    ✨ Using an output folder is the SAFEST option as it leaves original files untouched.
#    If this is set to None, the script will rename the original files IN-PLACE.
#    Example: r"C:\path\to\your\output_folder"
output_folder = r"F:\OneDrive - Green Energy\Desktop\samples\Flutter\texts" # <--- SET YOUR OUTPUT FOLDER

# 3. Set to True to process files in all subfolders, or False to only process
#    files in the top-level folder.
include_subfolders = True

# 4. List of file extensions to INCLUDE in the conversion.
#    Only files matching these patterns will be considered for processing.
#    Uses wildcard matching (e.g., "*.dart" matches any file ending in .dart).
#    Set to ["*"] to include all files by default (before applying exclusions).
# included_extensions = ["*.dart", "*.yaml", "*.md", "*.json", "*.xml"] 
included_extensions = ["*.dart"] 

# 5. List of file extensions to EXCLUDE from conversion.
#    This runs AFTER the inclusion check. If a file is included, this can still
#    exclude it.
excluded_extensions = ["*.wav", "*.mp4", "*.exe", "*.jpg", "*.png"]

# 6. List of folder names to completely skip during processing.
#    If a folder's name is in this list, neither it nor any of its
#    subfolders will be processed.
excluded_folders = ["venv", ".git", "temp_files", "__pycache__", ".idea", "build"]

# ==============================================================================
# SCRIPT LOGIC - NO NEED TO EDIT BELOW THIS LINE
# ==============================================================================

def convert_files_in_folder(root_path, output_path, process_subfolders, ext_inclusions, ext_exclusions, folder_exclusions):
    """
    Copies or renames files from a source directory, converting them to .txt.
    Includes options for recursion, inclusion/exclusion filters, and folder skipping.
    If output_path is None, it renames files in-place.
    """
    if not os.path.isdir(root_path):
        print(f"❌ Error: The source folder '{root_path}' does not exist. Please check the path.")
        return

    # Determine the mode of operation
    if output_path:
        mode = 'copy'
        os.makedirs(output_path, exist_ok=True) # Ensure the output folder exists
        print(f"🚀 Starting process in 'Copy Mode'.")
        print(f"   Source: '{root_path}'")
        print(f"   Destination: '{output_path}'")
    else:
        mode = 'rename'
        print(f"🚀 Starting process in 'Rename In-Place Mode'.")
        print(f"   ⚠️ Warning: Files in '{root_path}' will be permanently renamed.")
    
    print("-" * 50)

    processed_count = 0
    skipped_count = 0
    warning_count = 0
    
    for dirpath, dirnames, filenames in os.walk(root_path):
        
        # Feature: Exclude specific folders from traversal
        dirnames[:] = [d for d in dirnames if d not in folder_exclusions]

        for filename in filenames:
            original_file_path = os.path.join(dirpath, filename)
            
            # --- Inclusion/Exclusion Checks ---
            if filename.endswith('.txt'):
                continue

            # 1. Check for inclusion: Must match one of the included patterns
            is_included = any(fnmatch.fnmatch(filename.lower(), pattern.lower()) for pattern in ext_inclusions)
            if not is_included:
                skipped_count += 1
                continue # Skip if not in the inclusion list

            # 2. Check for exclusion: Must NOT match any excluded patterns
            is_excluded = any(fnmatch.fnmatch(filename.lower(), pattern.lower()) for pattern in ext_exclusions)
            if is_excluded:
                # This check is silent to avoid cluttering the output.
                skipped_count += 1
                continue # Skip if in the exclusion list

            # --- Path and Naming Logic ---
            new_filename = f"{filename}.txt"
            
            try:
                if mode == 'copy':
                    # Recreate the directory structure in the output folder
                    relative_dir = os.path.relpath(dirpath, root_path)
                    output_dir = os.path.join(output_path, relative_dir)
                    os.makedirs(output_dir, exist_ok=True)
                    
                    destination_path = os.path.join(output_dir, new_filename)
                    action_verb = "Copied"
                    
                    if os.path.exists(destination_path):
                        print(f"⚠️ Warning: '{destination_path}' already exists. Skipping copy.")
                        warning_count += 1
                        continue

                    shutil.copy2(original_file_path, destination_path)

                else: # mode == 'rename'
                    destination_path = os.path.join(dirpath, new_filename)
                    action_verb = "Renamed"

                    if os.path.exists(destination_path):
                        print(f"⚠️ Warning: '{destination_path}' already exists. Skipping rename.")
                        warning_count += 1
                        continue
                        
                    os.rename(original_file_path, destination_path)

                print(f"✅ {action_verb}: '{filename}' -> '{new_filename}'")
                processed_count += 1

            except (OSError, shutil.Error) as e:
                print(f"❌ Error processing '{filename}': {e}")
                warning_count += 1

        if not process_subfolders:
            break
            
    print("-" * 50)
    print("🎉 Process Complete!")
    print(f"   - Files Processed: {processed_count}")
    print(f"   - Files Skipped (due to filters): {skipped_count}")
    print(f"   - Warnings/Errors: {warning_count}")

# --- Execute the Function ---
convert_files_in_folder(
    root_path=target_folder,
    output_path=output_folder,
    process_subfolders=include_subfolders,
    ext_inclusions=included_extensions,
    ext_exclusions=excluded_extensions,
    folder_exclusions=excluded_folders
)

🚀 Starting process in 'Copy Mode'.
   Source: 'F:\OneDrive - Green Energy\Desktop\samples\Flutter'
   Destination: 'F:\OneDrive - Green Energy\Desktop\samples\Flutter\texts'
--------------------------------------------------
✅ Copied: 'main.dart' -> 'main.dart.txt'
✅ Copied: 'supabase_client.dart' -> 'supabase_client.dart.txt'
✅ Copied: 'leaderboard_entry_data.dart' -> 'leaderboard_entry_data.dart.txt'
✅ Copied: 'models.dart' -> 'models.dart.txt'
✅ Copied: 'quiz_data.dart' -> 'quiz_data.dart.txt'
✅ Copied: 'quiz_question.dart' -> 'quiz_question.dart.txt'
✅ Copied: 'app.dart' -> 'app.dart.txt'
✅ Copied: 'ai_chat_provider.dart' -> 'ai_chat_provider.dart.txt'
✅ Copied: 'exam_status_provider.dart' -> 'exam_status_provider.dart.txt'
✅ Copied: 'history_provider.dart' -> 'history_provider.dart.txt'
✅ Copied: 'quiz_cache_provider.dart' -> 'quiz_cache_provider.dart.txt'
✅ Copied: 'settings_provider.dart' -> 'settings_provider.dart.txt'
✅ Copied: 'theme_provider.dart' -> 'theme_provider.dart.txt